In [ ]:
# 3) Since the times are increasing monotonically by 1/10,000 second in our training data, find the begin and end row numbers to black out.
# Test blackout first on beginning of 7, 8.3, and 9 m/s and compare with what we get using threshold method.
# Then test at 18, 19, 20m/s on multidf data.

In [48]:
import pickle
import time
import os
import datetime
import numpy as np
import pandas as pd

cur_dir = os.getcwd()
main_dir = os.path.dirname(os.path.dirname(cur_dir))
offsets_dir = os.path.join(cur_dir, 'offset_pickles')

In [49]:
test_dates = ["08_01_2022","08_02_2022"]
test_folders = ["Day1_Training1", "Day2_Training1"]
file_names = ["compensated_normalized_WTRUN2_training_sweep1_2022-08-01_17-24-43-50_rtd-str_uniqueRTDs_infrequent", 
              "compensated_normalized_WTRUN2_day2_training1_2022-08-02_12-38-30-01_rtd-str_uniqueRTDs_infrequent"]

In [4]:
test_dfs = list()
for i in range(len(test_folders)):
  data_dir = os.path.join(main_dir, test_dates[i]+"_Tests", "testdata", test_folders[i])
  test_csv = os.path.join(data_dir, file_names[i]+".csv")
  test_df = pd.read_csv(test_csv, header=0)
  test_dfs.append(test_df)

In [50]:
# Load the RTD/SG vs EDS offsets in case we need to shift timing in case RTD/SG is not the vantage data
with open(os.path.join(offsets_dir, "eds_rtdstr_offsets.pkl"),'rb') as f:
  start_loc_offsets = pickle.load(f) #[['vantage test 1 (eds or rtdstr)', vantage offset from other data source 1], ['vantage test 2 (eds or rtdstr)', vantage offset from other data source 2]]

In [53]:
start_loc_offsets

[['eds', 1198], ['eds', 1036]]

In [5]:
# Convert the datetime column to usable DateTime Object.
for test_df in test_dfs:
  test_df["rtdstr_DateTime Obj"] = test_df["Date/Time"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d_%H-%M-%S-%f"))

In [111]:
# Get the RTD excitation times on each rtd-str df (non-combined or non-repeated ones).
blackout_masks_all = dict()

for i, test_df in enumerate(test_dfs):
  if start_loc_offsets[i][0] != "rtdstr":
    start_time = test_df["rtdstr_DateTime Obj"][start_loc_offsets[i][1]]
  else:
    start_time = test_df["rtdstr_DateTime Obj"][0]
  end_time = test_df["rtdstr_DateTime Obj"].iloc[-1]
  excitation_times = test_df.loc[::100, "rtdstr_DateTime Obj"]
  
  blackout_times = np.empty((excitation_times.shape[0], 2), dtype="datetime64[ns]")
  blackout_times[:,0] = excitation_times.to_numpy() - np.timedelta64(40, 'ms') #0.04 second
  blackout_times[:,1] = excitation_times.to_numpy()
  
  blackout_lines = np.zeros((excitation_times.shape[0], 2))
  blackout_lines[:,0] = (blackout_times[:,0] - start_time.to_numpy()).astype('timedelta64[ms]') * 10
  blackout_lines[:,1] = (blackout_times[:,1] - start_time.to_numpy()).astype('timedelta64[ms]') * 10
  ix = np.where(blackout_lines>0)
  blackout_lines = blackout_lines[ix[0][0]:]

  ms_elapsed = (end_time.to_numpy() - start_time.to_numpy()).astype('timedelta64[ms]') / np.timedelta64(1, 'ms')
  total_num_lines = ms_elapsed * 10
  blackout_masks = np.full(int(total_num_lines), True)
  for j in range(blackout_lines.shape[0]):
    blackout_pair = blackout_lines[j]
    if blackout_pair[0] < 0:
      blackout_pair[0] = 0
    blackout_masks[int(blackout_pair[0]) : int(blackout_pair[1])] = False
    blackout_masks[0 : 10] = False

  blackout_masks_all[test_folders[i]] = blackout_masks


In [118]:
for i,test_folder in enumerate(test_folders):
  data_dir = os.path.join(main_dir, test_dates[i]+"_Tests", "testdata", test_folder)
  filename = "blackout_masks"
  np.save(os.path.join(data_dir, filename), blackout_masks_all[test_folder])